In [1]:
##Media data raw
##Target data raw(all three)
##Non-subs data
##Search - all three and normalized

In [2]:
library(tidyverse)

Warning message:
"package 'tidyverse' was built under R version 3.6.3"-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.3.1     v purrr   0.3.2
v tibble  3.0.4     v dplyr   1.0.2
v tidyr   1.1.0     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"Warning message:
"package 'forcats' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [3]:
load('../B2C_Acquisition_Q321_refresh/RData/data_XyZ_ts_comp.RData')
#load('RData/data_XyZ_ts_sales.RData')
load('../B2C_Acquisition_Q321_refresh/RData/pp_MMM_ts_comp_xy995.RData')

In [4]:
#updated_ga_data <- read.csv('Data/mmm_b2c_acq_opt_sdl_combo_Jan18_Mar20_20210309.csv',stringsAsFactors = F)
non_sub_data <- read.csv('../B2C_Acquisition_Q321_refresh/Data/non_sub_data.csv')

In [5]:
ls()

[1] "data_XyZ_ts_comp"       "df_causal"              "df_corr"               
 [4] "df_rc1"                 "df_rc2"                 "df_rc3"                
 [7] "df_rc4"                 "df_ri1"                 "df_ri2"                
[10] "df_ri3"                 "df_ri4"                 "df_sbc_altice"         
[13] "df_sbc_brand"           "df_sbc_dma"             "df_spending"           
[16] "extract_fixed_effects"  "extract_random_coefs"   "extract_random_effects"
[19] "feature_levels"         "fit_lm1"                "fit_lm2"               
[22] "fit_lm3"                "fit_lm4"                "fit_rc1"               
[25] "fit_rc2"                "fit_rc3"                "fit_rc4"               
[28] "fit_ri1"                "fit_ri2"                "fit_ri3"               
[31] "fit_ri4"                "get_coef"               "gg_theme"              
[34] "lf1"                    "lf2"                    "lf3"                   
[37] "lf4"                    "non_sub_data"           "pair_cols_ls"          
[40] "pair_plot_OPT"          "pair_plot_SDL"          "plot_density"          
[43] "plot_scatter"           "pp_MMM_ts_comp_xy995"   "rc1"                   
[46] "rc2"                    "rc3"                    "rc4"                   
[49] "ri1"                    "ri2"                    "ri3"                   
[52] "ri4"                    "target_var"

In [6]:
glimpse(data_XyZ_ts_comp)

Rows: 5,850
Columns: 92
$ brand                       <chr> "Suddenlink", "Suddenlink", "Suddenlink...
$ dma                         <chr> "ABILENE-SWEETWATER TX", "ABILENE-SWEET...
$ period                      <date> 2017-12-31, 2018-01-07, 2018-01-14, 20...
$ gross_add_true              <dbl> 212, 235, 188, 180, 201, 181, 171, 176,...
$ gross_add_tot               <dbl> 285, 294, 249, 235, 290, 255, 253, 246,...
$ dm_volume                   <dbl> 26044.42, 19237.14, 19212.19, 17909.49,...
$ dm_spend                    <dbl> 7079.970, 5466.984, 5355.782, 5041.937,...
$ audio_spend                 <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...
$ video_spend                 <dbl> 249.33, 199.27, 199.62, 187.03, 222.08,...
$ display_spend               <dbl> 1187.00, 1193.94, 2237.93, 2454.55, 301...
$ paid_search_spend_brand     <dbl> 636.31, 1080.04, 1084.88, 1070.57, 1069...
$ paid_search_spend_unbranded <dbl> 1110.56, 1878.54, 865.93, 417.30, 549.3...
$ paid_search_spend         

In [7]:
media_names_vec <- c("radio_spend","tv_spend","print_spend","dm_spend","audio_spend", 
                      "display_spend","video_spend","paid_search_spend_brand", 
                      "paid_search_spend_unbranded","social_spend")

In [8]:
raw_data <- data_XyZ_ts_comp %>% 
#select(-matches('google'),-'gross_add_true') %>% 
#left_join(sales_data,by=c('dma','period')) %>%
#left_join(ga_data,by = c('dma','period')) %>%
         select(period,dma,media_names_vec,matches('google*'),gross_add_true) 


Note: Using an external vector in selections is ambiguous.
i Use `all_of(media_names_vec)` instead of `media_names_vec` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.


In [9]:
glimpse(raw_data)

Rows: 5,850
Columns: 16
$ period                      <date> 2017-12-31, 2018-01-07, 2018-01-14, 20...
$ dma                         <chr> "ABILENE-SWEETWATER TX", "ABILENE-SWEET...
$ radio_spend                 <dbl> 3156, 3156, 3156, 1052, 3156, 2582, 258...
$ tv_spend                    <dbl> 7409.00, 6356.50, 7988.50, 7834.25, 741...
$ print_spend                 <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.00...
$ dm_spend                    <dbl> 7079.970, 5466.984, 5355.782, 5041.937,...
$ audio_spend                 <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.0...
$ display_spend               <dbl> 1187.00, 1193.94, 2237.93, 2454.55, 301...
$ video_spend                 <dbl> 249.33, 199.27, 199.62, 187.03, 222.08,...
$ paid_search_spend_brand     <dbl> 636.31, 1080.04, 1084.88, 1070.57, 1069...
$ paid_search_spend_unbranded <dbl> 1110.56, 1878.54, 865.93, 417.30, 549.3...
$ social_spend                <dbl> 391, 528, 339, 327, 580, 798, 903, 942,...
$ google_altice_search_idx  

In [10]:
raw_data %>%
group_by(dma) %>%
summarize(max(google_altice_search_idx))

`summarise()` ungrouping output (override with `.groups` argument)


dma,max(google_altice_search_idx)
<chr>,<dbl>
ABILENE-SWEETWATER TX,39419.46
ALEXANDRIA LA,59564.32
All Other Suddenlink DMA Total,120725.41
AMARILLO TX,62894.05
AUSTIN TX,52851.89
BLUEFIELD-BECKLEY-OAK HILL WV,46775.14
CHARLESTON-HUNTINGTON WV,95358.92
DALLAS-FT WORTH TX,118492.97
EUREKA CA,27038.92


In [11]:
pp_MMM_ts_comp_xy995$control$Z_minmax %>%
filter(X == 'seasonality_gross_add_true')

dma_id,dma,X,X_min,X_max
<int>,<chr>,<chr>,<dbl>,<dbl>
1,ABILENE-SWEETWATER TX,seasonality_gross_add_true,-0.0013106478,0.0022322113
2,ALEXANDRIA LA,seasonality_gross_add_true,-0.0011723822,0.0014766895
3,All Other Suddenlink DMA Total,seasonality_gross_add_true,-0.0006571787,0.0020171003
4,AMARILLO TX,seasonality_gross_add_true,-0.0009381807,0.0009805729
5,AUSTIN TX,seasonality_gross_add_true,-0.0015529935,0.0012379408
6,BLUEFIELD-BECKLEY-OAK HILL WV,seasonality_gross_add_true,-0.0010069328,0.0007347747
7,CHARLESTON-HUNTINGTON WV,seasonality_gross_add_true,-0.0009558329,0.0007870307
8,DALLAS-FT WORTH TX,seasonality_gross_add_true,-0.0006020922,0.0006484964
9,EUREKA CA,seasonality_gross_add_true,-0.0023507966,0.0053403357


In [12]:
norm_comp_data <- pp_MMM_ts_comp_xy995$control$Z_per_capita %>%
                  select(period,dma,matches('google'))
norm_comp_data

period,dma,google_altice_search_idx,google_comp_search_idx,google_non_brand_search_idx
<date>,<chr>,<dbl>,<dbl>,<dbl>
2017-12-31,ABILENE-SWEETWATER TX,0.0000000,0.0000000,0.0000000
2018-01-07,ABILENE-SWEETWATER TX,0.0000000,0.0000000,0.0000000
2018-01-14,ABILENE-SWEETWATER TX,0.0000000,0.0000000,0.0000000
2018-01-21,ABILENE-SWEETWATER TX,0.0000000,0.0000000,0.0000000
2018-01-28,ABILENE-SWEETWATER TX,0.0000000,0.0000000,0.0000000
2018-02-04,ABILENE-SWEETWATER TX,0.4279133,0.5366166,0.5635207
2018-02-11,ABILENE-SWEETWATER TX,0.4355922,0.5173447,0.5260855
2018-02-18,ABILENE-SWEETWATER TX,0.4557498,0.5854387,0.7240145
2018-02-25,ABILENE-SWEETWATER TX,0.4774429,0.5751604,0.6825966


In [13]:
non_sub_data <- non_sub_data 

In [14]:
##In Q3 2021 refresh the intial periods of google search index was missing. So restricting the data to the latest periods when it was available
raw_data  <-   raw_data %>%
               filter(google_altice_search_idx != 0 ) 
raw_data

period,dma,radio_spend,tv_spend,print_spend,dm_spend,audio_spend,display_spend,video_spend,paid_search_spend_brand,paid_search_spend_unbranded,social_spend,google_altice_search_idx,google_comp_search_idx,google_non_brand_search_idx,gross_add_true
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018-02-04,ABILENE-SWEETWATER TX,2582.00,8986.75,0.000,5859.819,0.00,2820.38,246.23,1050.05,828.44,798.00,16868.11,9482.16,10708.11,181
2018-02-11,ABILENE-SWEETWATER TX,2582.00,9306.50,0.000,4209.766,0.00,2709.71,241.35,995.82,743.64,903.00,17170.81,9141.62,9996.76,171
2018-02-18,ABILENE-SWEETWATER TX,2582.00,9860.50,3761.218,5230.298,0.00,2872.46,921.29,1014.38,1212.05,942.00,17965.41,10344.86,13757.84,176
2018-02-25,ABILENE-SWEETWATER TX,2682.00,12698.75,0.000,4654.164,0.00,3118.46,107.03,735.53,1291.98,937.00,18820.54,10163.24,12970.81,288
2018-03-04,ABILENE-SWEETWATER TX,3038.00,11186.00,0.000,4901.217,0.00,3309.93,519.44,896.76,2047.30,875.00,16595.68,9232.43,11275.68,255
2018-03-11,ABILENE-SWEETWATER TX,0.00,10561.75,0.000,6419.880,48.05,2882.42,4051.79,1023.59,3143.89,701.00,17049.73,9572.97,11472.43,210
2018-03-18,ABILENE-SWEETWATER TX,0.00,10648.00,0.000,7125.820,257.30,3176.66,2916.52,1033.26,4521.01,539.00,15725.41,9058.38,9580.54,218
2018-03-25,ABILENE-SWEETWATER TX,788.00,7297.00,0.000,7179.777,266.89,3007.42,2689.58,1074.96,2726.40,298.00,16398.92,8899.46,9565.41,164
2018-04-01,ABILENE-SWEETWATER TX,763.00,9133.00,0.000,6337.543,6.94,2883.08,397.48,1119.62,1780.41,414.00,16860.54,8854.05,11230.27,192


In [15]:
final_obj = list('Raw_data' = raw_data,
                 'Non_Sub_data' = non_sub_data,
                 'Norm_Comp_data' = norm_comp_data)

In [16]:
save(final_obj,file = 'RawData/mmm_b2c_acq_q321_refresh_raw_data.RData')

In [21]:
load('RawData/mmm_b2c_acq_q321_refresh_raw_data.RData')

In [22]:
ls()

[1] "data_XyZ_ts_comp"       "df_causal"              "df_corr"               
 [4] "df_rc1"                 "df_rc2"                 "df_rc3"                
 [7] "df_rc4"                 "df_ri1"                 "df_ri2"                
[10] "df_ri3"                 "df_ri4"                 "df_sbc_altice"         
[13] "df_sbc_brand"           "df_sbc_dma"             "df_spending"           
[16] "extract_fixed_effects"  "extract_random_coefs"   "extract_random_effects"
[19] "feature_levels"         "final_obj"              "fit_lm1"               
[22] "fit_lm2"                "fit_lm3"                "fit_lm4"               
[25] "fit_rc1"                "fit_rc2"                "fit_rc3"               
[28] "fit_rc4"                "fit_ri1"                "fit_ri2"               
[31] "fit_ri3"                "fit_ri4"                "get_coef"              
[34] "gg_theme"               "lf1"                    "lf2"                   
[37] "lf3"                    "lf4"                    "media_names_vec"       
[40] "non_sub_data"           "norm_comp_data"         "pair_cols_ls"          
[43] "pair_plot_OPT"          "pair_plot_SDL"          "plot_density"          
[46] "plot_scatter"           "pp_MMM_ts_comp_xy995"   "raw_data"              
[49] "rc1"                    "rc2"                    "rc3"                   
[52] "rc4"                    "ri1"                    "ri2"                   
[55] "ri3"                    "ri4"                    "target_var"

In [23]:
names(final_obj)

[1] "Raw_data"       "Non_Sub_data"   "Norm_Comp_data"

In [24]:
final_obj$Raw_data

period,dma,radio_spend,tv_spend,print_spend,dm_spend,audio_spend,display_spend,video_spend,paid_search_spend_brand,paid_search_spend_unbranded,social_spend,google_altice_search_idx,google_comp_search_idx,google_non_brand_search_idx,gross_add_true
<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018-02-04,ABILENE-SWEETWATER TX,2582.00,8986.75,0.000,5859.819,0.00,2820.38,246.23,1050.05,828.44,798.00,16868.11,9482.16,10708.11,181
2018-02-11,ABILENE-SWEETWATER TX,2582.00,9306.50,0.000,4209.766,0.00,2709.71,241.35,995.82,743.64,903.00,17170.81,9141.62,9996.76,171
2018-02-18,ABILENE-SWEETWATER TX,2582.00,9860.50,3761.218,5230.298,0.00,2872.46,921.29,1014.38,1212.05,942.00,17965.41,10344.86,13757.84,176
2018-02-25,ABILENE-SWEETWATER TX,2682.00,12698.75,0.000,4654.164,0.00,3118.46,107.03,735.53,1291.98,937.00,18820.54,10163.24,12970.81,288
2018-03-04,ABILENE-SWEETWATER TX,3038.00,11186.00,0.000,4901.217,0.00,3309.93,519.44,896.76,2047.30,875.00,16595.68,9232.43,11275.68,255
2018-03-11,ABILENE-SWEETWATER TX,0.00,10561.75,0.000,6419.880,48.05,2882.42,4051.79,1023.59,3143.89,701.00,17049.73,9572.97,11472.43,210
2018-03-18,ABILENE-SWEETWATER TX,0.00,10648.00,0.000,7125.820,257.30,3176.66,2916.52,1033.26,4521.01,539.00,15725.41,9058.38,9580.54,218
2018-03-25,ABILENE-SWEETWATER TX,788.00,7297.00,0.000,7179.777,266.89,3007.42,2689.58,1074.96,2726.40,298.00,16398.92,8899.46,9565.41,164
2018-04-01,ABILENE-SWEETWATER TX,763.00,9133.00,0.000,6337.543,6.94,2883.08,397.48,1119.62,1780.41,414.00,16860.54,8854.05,11230.27,192
